In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import torch as torch
import os
import torch.optim as optim
import torch.nn.functional as F

In [2]:
DATA_DIRECTORY = "../datos/"

In [3]:
diagnosticos  = pd.read_excel(DATA_DIRECTORY+"RESUMEN TAC CEREBRALES.xlsx")

In [4]:
# diccionario cuya llave es el id de paciente y el valor una lista 
# donde cada elemento de la lista es la matriz de una i
diccionario_imagenes_pacientes = dict()

for paciente in diagnosticos.paciente:
    directorio_paciente = DATA_DIRECTORY+"paciente_"+str(paciente)
    archivos_paciente = os.listdir(directorio_paciente)
    
    lista_imagenes_paciente = []
    for archivo in archivos_paciente:
        if archivo.endswith(".jpg"):
            imagen = mpimg.imread(directorio_paciente+"/"+archivo)
            lista_imagenes_paciente.append(imagen)
            
    diccionario_imagenes_pacientes[paciente] = lista_imagenes_paciente
    

## Modelos y arquitecturas
### Arquitecturas experimental  DNC
* Alimentamos al modelo imagen por imagen y se presenta un solo diagnostico por paciente
* El controller de la DNC esta compuesto por una convnet

In [5]:
CONTROLLER_OUTPUT_SIZE = 1024

In [6]:
#TODO: cambiar valores quemados por valores parametrizados y calculos dependientes
class ConvController(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(1,4,kernel_size=3,stride=1)
        self.fc1  =  torch.nn.Linear(262144,CONTROLLER_OUTPUT_SIZE)
        
    def forward(self,x):
        h = self.conv1(x)
        
        #flatten
        h =  x.view(-1,x.shape[1]*x.shape[2]*x.shape[3])
        h =  self.fc1(h)
        
        return h

In [7]:
#TODO: cambiar valores quemados por valores parametrizados y calculos dependientes
#TODO: cordar por que en algun momento le puse bias = False a los pesos del vector de salida de la DNC
class DNC(torch.nn.Module):
    
    def __init__(self,controller,memory_size = (10,10),read_heads = 1):
        super().__init__()
        self.controller = controller
        self.N = memory_size[0] # number of memory locations
        self.W = memory_size[1] # word size of the memory 
        self.R = read_heads # number of read heads
        self.WS = 1 #not in the paper(they use 1), but used as a parametrizable number of write heads for further experiments
        self.interface_vector_size = (self.W*self.R) + (self.W*self.WS) + (2*self.W) + (5*self.R) + 3
        
        #1024 es el tamaño del vector de salida del controlador, 1 es el tamaño de salida de la dnc
        self.output_vector_linear = torch.nn.Linear(CONTROLLER_OUTPUT_SIZE,1,bias=False) #W_y 
        self.interface_vector_linear = torch.nn.Linear(CONTROLLER_OUTPUT_SIZE,self.interface_vector_size,bias=False) #W_ξ
        
        self.read_keys = torch.Tensor(size=(self.R,self.W)).requires_grad_(False)
        
    def forward(self,x):
        h_t = self.controller(x) #controller output called ht in the paper
        
        output_vector = self.output_vector_linear(h_t) # called Vt in the paper(υ=Wy[h1;...;hL]) v_o_t in my txt
        interface_vector = self.interface_vector_linear(h_t) #called ξt(ksi) in the paper ,ξ_t in my txt
        
        self.read_keys.data = interface_vector[0,0:self.R*self.W].view((self.R,self.W)).data
        print(self.read_keys.size())
        
        y_t = F.sigmoid(output_vector)
        return y_t

## Experimentos
* Experimentando con DNC alimentando una imagen a la vez en orden aleatorio con pacientes también en orden aleatorio

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
EPOCHS = 2

In [10]:
conv_controller = ConvController()
dnc_model = DNC(controller=conv_controller,read_heads=2).to(device)

In [11]:
def loss_function(y,y_hat,last_flag):
    #print(y,y_hat,last_flag)
    base_criterion = torch.nn.BCELoss()
    return torch.full_like(y,last_flag) * base_criterion(y,y_hat)

In [12]:
criterion = loss_function
optimizer = optim.Adam(dnc_model.parameters(),lr=0.001)

In [13]:
for epoch in range(EPOCHS):
    # en cada epoch procesar los pacientes en orden aleatorio
    pacientes = np.random.choice(np.array(diagnosticos.paciente),size= len(diagnosticos.paciente),replace=False)
    
    for paciente in pacientes:
        imagenes_paciente = diccionario_imagenes_pacientes.get(paciente)
        diagnostico_hemorragia_paciente = np.array(float(diagnosticos[diagnosticos.paciente==paciente].hemorragia))
        tensor_diagnostico_hemorragia_paciente = torch.Tensor(diagnostico_hemorragia_paciente).to(device)
        
        indices_imagenes_pacientes = np.arange(0,len(imagenes_paciente)-1,step=1)
        indices_aleatorios_imagenes = np.random.choice(indices_imagenes_pacientes,len(indices_imagenes_pacientes),replace=False)
        
        for indice in indices_aleatorios_imagenes:
            last_image =  int(indice  == indices_aleatorios_imagenes[-1])
            
            optimizer.zero_grad()
            
            imagen_paciente = imagenes_paciente[indice]
            tensor_imagen_paciente =  torch.unsqueeze(
                torch.unsqueeze( torch.Tensor(imagen_paciente),dim=0),dim=1).to(device)
            
            print("Alimentando paciente {} e imagen {} al modelo".format(paciente,indice))
            
            diagnostico_hemorragia_aproximado = dnc_model(tensor_imagen_paciente)
            
            loss = criterion(diagnostico_hemorragia_aproximado,tensor_diagnostico_hemorragia_paciente,last_image)
            loss.backward()
            optimizer.step()
            
            if last_image:
                print(loss.cpu().data,diagnostico_hemorragia_aproximado.cpu().data)

Alimentando paciente 1 e imagen 9 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 23 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 6 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 18 al modelo
torch.Size([2, 10])


/home/luis/anaconda2/envs/pytorch_challenge/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/luis/anaconda2/envs/pytorch_challenge/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Alimentando paciente 1 e imagen 8 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 1 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 0 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 16 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 17 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 7 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 2 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 13 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 12 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 21 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 11 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 14 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 3 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 4 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 19 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 

Alimentando paciente 4 e imagen 9 al modelo
torch.Size([2, 10])
Alimentando paciente 4 e imagen 2 al modelo
torch.Size([2, 10])
Alimentando paciente 4 e imagen 13 al modelo
torch.Size([2, 10])
Alimentando paciente 4 e imagen 4 al modelo
torch.Size([2, 10])
Alimentando paciente 4 e imagen 11 al modelo
torch.Size([2, 10])
Alimentando paciente 4 e imagen 3 al modelo
torch.Size([2, 10])
Alimentando paciente 4 e imagen 5 al modelo
torch.Size([2, 10])
Alimentando paciente 4 e imagen 1 al modelo
torch.Size([2, 10])
Alimentando paciente 4 e imagen 12 al modelo
torch.Size([2, 10])
Alimentando paciente 4 e imagen 0 al modelo
torch.Size([2, 10])
tensor([[27.6310]]) tensor([[1.]])
Alimentando paciente 1 e imagen 3 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 22 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 8 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 6 al modelo
torch.Size([2, 10])
Alimentando paciente 1 e imagen 10 al modelo
torch.Size([2, 10])


In [14]:
#TODO: averiguar por que salen 6 tensores de parametros si solo se han declarado 3(al momento de correr lap rueba)
train_parmams = list(dnc_model.named_parameters())

for train_param in train_parmams:
    print(train_param[0])

controller.conv1.weight
controller.conv1.bias
controller.fc1.weight
controller.fc1.bias
output_vector_linear.weight
interface_vector_linear.weight


In [15]:
dnc_model.read_keys.data


tensor([[-10364.4141,  18726.9434, -28569.4395,  14022.5586, -11180.3438,
         -16211.1592,  20953.8320,  22694.6445,  -1802.7400,   5784.9512],
        [  7135.8345, -14423.8984, -13506.4863,  28326.8320, -52403.8477,
         -65830.2656,  -5733.7036,   4240.8174,    737.7520,  10023.2471]],
       device='cuda:0')